In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
# setup training datasets

train_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    transform=ToTensor(),
    download=True,
    target_transform=None
)

In [ ]:
# set the class name and indexes

image, label= train_data[0]
class_name= train_data.classes
idx=train_data.class_to_idx
image,label,class_name

In [ ]:
image.shape, class_name[label]

In [ ]:
#plot the images of data
torch.manual_seed(42)
fig=plt.figure(figsize=(9,9))

rows,cols=4,4

#Now make a for loop
for i in range(1,rows*cols+1):
    random_idx= torch.randint(0,len(train_data),size=[1]).item()
    img,label=train_data[random_idx]
    fig.add_subplot(rows,cols,i)
    plt.imshow(img.squeeze(),cmap='gray')
    plt.title(class_name[label])
    plt.axis(False)

### Prepare the Dataloader


In [ ]:
from torch.utils.data import DataLoader

batch_size=32

train_data_loader=DataLoader(dataset=train_data, batch_size= batch_size,shuffle=True)

test_data_loader=DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
# Lets checkout 
print(f'DataLoader: {train_data_loader, test_data_loader}')
print(f'Length of Train Data Loaders : {len(train_data_loader),len(test_data_loader)}')

In [ ]:
train_feature_batch, train_label_batch = next(iter(train_data_loader))
train_feature_batch,train_label_batch

In [ ]:
# Showing a sample
torch.manual_seed(42)
random_idx=torch.randint(0,len(train_feature_batch),size=[1]).item()
img,label=train_feature_batch[random_idx],train_label_batch[random_idx]
plt.imshow(img.squeeze())
plt.title(class_name[label])
plt.axis(False)

### Make a model

In [81]:
# Device agnostic code
device= 'cuda' if torch.cuda.is_available() else 'cpu'

class FashionMNISTModel(nn.Module):
    def __init__(self, 
                 input_shape: int,
                 hidden_units:int,
                 output_shape:int):
        super().__init__()

        self.layer_stack=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape,out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )

    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)

In [91]:
torch.manual_seed(42)
torch.set_float32_matmul_precision('high')

model_0=torch.compile(FashionMNISTModel(input_shape=784,
                          hidden_units=10,
                          output_shape=len(class_name))).to(device)

### Setup loss, optimizer and evaluation metrics

In [92]:
import requests

request= requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py')
with open('helper_functions.py','wb') as f:
    f.write(request.content)

In [93]:
## We will use accuracy as evaluation metrics
from helper_functions import accuracy_fn

# Setup loss and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(params=model_0.parameters(),lr=.1)

### Making Training and Testing loop

In [ ]:
### Trainning loop function
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device =device):

    
    train_loss,train_acc=0,0
      #training
    model.train()
    
    # add a loop to loop through the training batch
    for X,y in data_loader:
        
        X,y= X.to(device),y.to(device)

        #forward pass
        y_pred=model(X)

        #Calculate the loss and accuracy
        loss =loss_fn(y_pred,y)
        train_loss +=loss
        train_acc +=accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

        # optimizer zero grad
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # ajust the training batch- divide total train loss by length of train dataloader
    train_loss /= len(data_loader)
    train_acc /=len(data_loader)

    print(f'Train_loss: {train_loss:.5f} | Train_acc: {train_acc:.5f}')


In [116]:
### testing loop function
def testing_step(model: torch.nn.Module,
                 data_loader:torch.utils.data.DataLoader,
                 loss_fn:torch.nn.Module,
                 accuracy_fn,
                 device=device):
    test_loss,test_acc=0,0
    model.eval()

    with torch.inference_mode():
        for X,y in data_loader:
            X,y= X.to(device),y.to(device)
            # forward pass
            test_pred=model(X)
            test_loss +=loss_fn(test_pred,y)
            test_acc +=accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # Calculate the test loss average per batch
        test_loss /= len(data_loader)

        # Calculate the test acc average per batch
        test_acc /=len(data_loader)

        #print out    
    print(f'Test loss: {test_loss :.5f}|Test acc: {test_acc:.5f}')

In [117]:
torch.manual_seed(42)
epochs=3

from tqdm.auto import tqdm

for epoch in tqdm(range(epochs)):
    print(f'\nEpoch: {epoch}\n------')
    
    train_step(model=model_0,
               data_loader=train_data_loader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device)
    
    testing_step(model=model_0,
                 data_loader=test_data_loader,
                 loss_fn=loss_fn,
                 accuracy_fn=accuracy_fn,
                 device=device)

  0%|          | 0/3 [00:00<?, ?it/s]


Epoch: 0
------
Train_loss: 0.63995 | Train_acc: 76.30000
Test loss: 0.65253|Test acc: 76.37780

Epoch: 1
------
Train_loss: 0.60848 | Train_acc: 77.66167
Test loss: 0.50035|Test acc: 82.50799

Epoch: 2
------
Train_loss: 0.42780 | Train_acc: 84.79833
Test loss: 0.45390|Test acc: 84.12540
